In [23]:
import tensorflow as tf
import numpy as np 
import pandas as pd
from sklearn import preprocessing


In [26]:
dfraw=np.loadtxt('/content/19664156-Audiobooks-data[UdemyIran.Com].csv', delimiter=',')
df_input= dfraw[:,1:-1]
targetsall=dfraw[:,-1]


In [68]:
sum_of_targets= int(np.sum(targetsall))
zero_counter= 0
deletelist=[]
for i in range(targetsall.shape[0]):
  if targetsall[1]==0:
    zero_counter +=1
    if zero_counter> sum_of_targets:
      deletelist.append(i)

unscaled_inputs= np.delete(df_input, deletelist, axis=0)
targets_equal_periors= np.delete(targetsall, deletelist, axis=0)




In [69]:
scaled_inputs= preprocessing.scale (unscaled_inputs)

In [70]:
scaled_inputs2= np.arange(scaled_inputs.shape[0])
np.random.shuffle(scaled_inputs2)

shuffled_inputs= scaled_inputs[scaled_inputs2]
shuffled_targets=targets_equal_periors[scaled_inputs2]

In [71]:
sample_count= scaled_inputs.shape[0]
train_samples=int(sample_count*0.8)
validate_sample=int(sample_count*0.1)
test_sample=sample_count - train_samples-validate_sample


In [72]:
train_inputs= shuffled_inputs[:train_samples]
train_targets=shuffled_targets[:train_samples]
validation_inputs= shuffled_inputs[train_samples:train_samples+validate_sample]
validation_targets= shuffled_targets[train_samples:train_samples+validate_sample]
test_inputs= shuffled_inputs[train_samples+validate_sample:]
test_targets=shuffled_targets[train_samples+validate_sample:]
print(np.sum(train_targets), train_samples)
print(np.sum(validation_targets), validate_sample)
print(np.sum(test_targets), test_sample)


428.0 1789
54.0 223
43.0 225


In [73]:
 np.savez('audiobooks_train', inputs= train_inputs, targets=train_targets)
 np.savez('audiobook_validation', inputs= validation_inputs, targets=validation_targets)
 np.savez('audiobook_test', inputs= test_inputs, targets= test_targets)

In [74]:
npz= np.load('/content/audiobooks_train.npz')
train_inputs= npz['inputs'].astype(np.float)
train_targets=npz['targets'].astype(np.int)

npz=np.load('/content/audiobook_validation.npz')
validation_inputs=npz['inputs'].astype(np.float)
validation_targets=npz['targets'].astype(np.int)

npz= np.load('/content/audiobook_test.npz')
test_inputs=npz['inputs'].astype(np.float)
test_targets=npz['targets'].astype(np.int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  T

In [75]:
input_size=10
output_size=2
hidden_layer_size=50
model=tf.keras.Sequential([tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                           tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                           tf.keras.layers.Dense(output_size, activation='softmax')
    
])
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy', metrics=['accuracy'] )

batch_size=50
max_epochs=50
early_stop=tf.keras.callbacks.EarlyStopping(patience=4)
model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stop],
          validation_data=(validation_inputs, validation_targets),
          verbose=1)


Epoch 1/50
36/36 [==============================] - 1s 7ms/step - loss: 0.5425 - accuracy: 0.7742 - val_loss: 0.4603 - val_accuracy: 0.8072
Epoch 2/50
36/36 [==============================] - 0s 3ms/step - loss: 0.4171 - accuracy: 0.8278 - val_loss: 0.3796 - val_accuracy: 0.8430
Epoch 3/50
36/36 [==============================] - 0s 3ms/step - loss: 0.3613 - accuracy: 0.8463 - val_loss: 0.3468 - val_accuracy: 0.8475
Epoch 4/50
36/36 [==============================] - 0s 3ms/step - loss: 0.3378 - accuracy: 0.8496 - val_loss: 0.3212 - val_accuracy: 0.8520
Epoch 5/50
36/36 [==============================] - 0s 4ms/step - loss: 0.3216 - accuracy: 0.8569 - val_loss: 0.3058 - val_accuracy: 0.8520
Epoch 6/50
36/36 [==============================] - 0s 3ms/step - loss: 0.3107 - accuracy: 0.8563 - val_loss: 0.2967 - val_accuracy: 0.8520
Epoch 7/50
36/36 [==============================] - 0s 3ms/step - loss: 0.3022 - accuracy: 0.8552 - val_loss: 0.2886 - val_accuracy: 0.8879
Epoch 8/50
36/36 [==

In [76]:
test_accuracy=model.evaluate(test_inputs, test_targets)

8/8 [==============================] - 0s 2ms/step - loss: 0.2821 - accuracy: 0.8933


In [77]:
print(test_accuracy)

[0.28206294775009155, 0.8933333158493042]
